In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

    
    
    
def Belief(State,p,r):
   
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;
            
        
def WhittleIndex(State,p,r):
    WhittleIndex = 0.;

    
    if State[0]== 0 and State[1]>0:
        l= State[1];
        x= Belief([0,l],p,r);
        y=Belief([0,l+1],p,r);
        WhittleIndex =((x-y)*(l+1)+y)/(1-p+(x-y)*l+y); 
    elif State[0] == 1:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
    
    else:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
            
    return WhittleIndex;

def update(S,C,Reward,alpha,p,r,q,s,gamma,N):
        #the update process of each timestep.
        
        #S is the list of belief states of all channels.
        #C is the list of actual channel states.
        W = [0.]*N;
        Y = [0.]*N;
        Action = [0]*N;
        Threshold = 0.;
        Seed = np.random.rand(N);
        
       #Get the Threshold. 
        for i in range(int(N*gamma)):
            Y[i]= WhittleIndex(S[i],p,r);
        for i in range(int(N*gamma),N):
            Y[i]= WhittleIndex(S[i],q,s);
        
        Y.sort();
        Threshold = Y[int((1-alpha)*N)];
        
        
        
       #Get the Action vector.
        for i in range(int(N*gamma)):
            W[i]= WhittleIndex(S[i],p,r);
        for i in range(int(N*gamma),N):
            W[i]= WhittleIndex(S[i],q,s);
    
        h=0;
        for i in range(N):
            if W[i]>= Threshold:
                Action[i]=1;
                h=h+1;
            if h >= alpha*N:
                break;
            
       #The update process.
       
        for i in range(int(N*gamma)):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<p)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<r)==True);
        
        for i in range(int(N*gamma),N):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<q)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<s)==True);
        
        
        
        return Reward;
    

In [2]:
def onetrial(S,C,Reward,timesteps,alpha,p,r,q,s,gamma,N):

    for i in range(timesteps):
        Reward = update(S,C,Reward,alpha,p,r,q,s,gamma,N);
    return Reward;

In [3]:
def average(mylist):
    sum = 0;
    
    for i in range(len(mylist)):
        sum = sum + mylist[i];
    
    return sum/len(mylist);

def variance(mylist):
    
    sum = 0;
    mean = average(mylist);
    
    for i in range(len(mylist)):
        sum = sum + (mylist[i]-mean)**2;
    sigma = 2*math.sqrt(sum)/len(mylist)
    
    return sigma;






In [6]:
import copy;

def mc(S,Reward,ntrials,timesteps,alpha,p,r,q,s,gamma,N):
    e_1 = r/(1-p+r);
    e_2 = s/(1-q+s);
    C = [0]*N;
    Seed = np.random.rand(N);
    for j in range(int(N*gamma)):
            C[j]= int((Seed[j]< e_1) == True);
            
    for j in range(int(N*gamma),N):
            C[j]= int((Seed[j]<e_2) == True);
   
    Reward = [0]*ntrials;
    
    for i in range(ntrials):
        
        SS= copy.deepcopy(S); 
        CC= copy.deepcopy(C);
        Reward[i] = onetrial(SS,CC,0,timesteps,alpha,p,r,q,s,gamma,N);
    
        
   
    
    
    
    
    return Reward;


In [7]:
print(mc([[0,0]]*100,0,10,500,0.3,0.75,0.2,0.8,0.3,0.6,100))

[10438, 10638, 10542, 10505, 10542, 10439, 10579, 10601, 10525, 10534]


In [15]:
import numpy as np;
from multiprocessing import Pool
def worker(n):
    return mc([[0,0]]*n,0,500,500,0.3,0.75,0.2,0.8,0.3,0.6,n);

num_processors = 16;
p=Pool(processes = num_processors);

t = list(range(5,505,25));

X = p.map(worker,t)[0];
E = p.map(worker,t)[1];

plt.errorbar(t,X,E);


NameError: name 'math' is not defined